# RFM ile Müşteri Segmentasyonu

In [31]:
import datetime as dt
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
df_ = pd.read_excel("online_retail_II.xlsx", sheet_name="Year 2010-2011")

In [32]:
df = df_.copy()

In [33]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.00,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.00,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.00,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.00,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.00,United Kingdom


In [34]:
df.shape

(541910, 8)

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541910 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      541910 non-null  object        
 1   StockCode    541910 non-null  object        
 2   Description  540456 non-null  object        
 3   Quantity     541910 non-null  int64         
 4   InvoiceDate  541910 non-null  datetime64[ns]
 5   Price        541910 non-null  float64       
 6   Customer ID  406830 non-null  float64       
 7   Country      541910 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [36]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,541910.00,9.55,218.08,-80995.00,1.00,3.00,10.00,80995.00
Price,541910.00,4.61,96.76,-11062.06,1.25,2.08,4.13,38970.00
Customer ID,406830.00,15287.68,1713.60,12346.00,13953.00,15152.00,16791.00,18287.00


In [37]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
Price               0
Customer ID    135080
Country             0
dtype: int64

In [38]:
df.dropna(inplace = True)

In [39]:
df.isnull().sum()

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
dtype: int64

In [40]:
df["Description"].nunique()

3896

In [41]:
df["Description"].value_counts()

WHITE HANGING HEART T-LIGHT HOLDER    2070
REGENCY CAKESTAND 3 TIER              1905
JUMBO BAG RED RETROSPOT               1662
ASSORTED COLOUR BIRD ORNAMENT         1418
PARTY BUNTING                         1416
                                      ... 
NEW BAROQUE B'FLY NECKLACE GREEN         1
SET OF 3 PINK FLYING DUCKS               1
RED ROSE AND LACE C/COVER                1
LILAC FEATHERS CURTAIN                   1
MINT DINER CLOCK                         1
Name: Description, Length: 3896, dtype: int64

In [42]:
df.groupby("Description").agg({"Quantity":"sum"}).sort_values("Quantity", ascending = False).head()

,Quantity
Description,
WORLD WAR 2 GLIDERS ASSTD DESIGNS,53215
JUMBO BAG RED RETROSPOT,45066
ASSORTED COLOUR BIRD ORNAMENT,35314
WHITE HANGING HEART T-LIGHT HOLDER,34147
PACK OF 72 RETROSPOT CAKE CASES,33409


In [43]:
df = df[~df["Invoice"].str.contains("C", na=False)]

In [44]:
df["TotalPrice"] = df["Quantity"] * df["Price"]

In [45]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.00,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.00,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.00,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.00,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.00,United Kingdom,20.34


RECENCY: Müşteri ya da kullanıcının etkileşime geçip işlem yaptığı son tarih ile analizde dikkate alınacak tarih arasındaki fark.
    Güncellik. Bu değerin küçük olması RFM analizinde daha değerli olduğu anlamına gelir.

FREQUENCY: Alışveriş, işlem sıklığı.

MONETARY: Toplam harcama.

In [46]:
today_date = dt.datetime(2011, 12, 11)

In [47]:
rfm = df.groupby('Customer ID').agg({'InvoiceDate': lambda InvoiceDate: (today_date - InvoiceDate.max()).days,
                                     'Invoice': lambda Invoice: Invoice.nunique(),
                                     'TotalPrice': lambda TotalPrice: TotalPrice.sum()})

In [48]:
rfm.head()

,InvoiceDate,Invoice,TotalPrice
Customer ID,,,
12346.00,326,1,77183.60
12347.00,3,7,4310.00
12348.00,76,4,1797.24
12349.00,19,1,1757.55
12350.00,311,1,334.40


In [49]:
rfm.columns = ['Recency', 'Frequency', 'Monetary']

In [50]:
rfm.head()

,Recency,Frequency,Monetary
Customer ID,,,
12346.00,326,1,77183.60
12347.00,3,7,4310.00
12348.00,76,4,1797.24
12349.00,19,1,1757.55
12350.00,311,1,334.40


In [53]:
rfm.shape

(4338, 3)

In [51]:
rfm = rfm[rfm["Monetary"] > 0]

In [54]:
rfm.shape

(4338, 3)

In [57]:
rfm["recency_score"] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1])

In [59]:
rfm["frequency_score"] = pd.qcut(rfm['Frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])

In [60]:
rfm["monetary_score"] = pd.qcut(rfm['Monetary'], 5, labels=[1, 2, 3, 4, 5])

In [61]:
rfm.head()

,Recency,Frequency,Monetary,recency_score,frequency_score,monetary_score
Customer ID,,,,,,
12346.00,326,1,77183.60,1,1,5
12347.00,3,7,4310.00,5,5,5
12348.00,76,4,1797.24,2,4,4
12349.00,19,1,1757.55,4,1,4
12350.00,311,1,334.40,1,1,2


In [62]:
rfm["RFM_SCORE"] = (rfm['recency_score'].astype(str) + rfm['frequency_score'].astype(str))

Monetary değerini dahil etmedik çünkü RFM analizinde işlem sayısı ve güncellik baskındır.

In [63]:
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

In [64]:
rfm['segment'] = rfm['RFM_SCORE'].replace(seg_map, regex=True)

In [66]:
rfm = rfm[["Recency", "Frequency", "Monetary", "segment"]]

In [67]:
rfm.head()

,Recency,Frequency,Monetary,segment
Customer ID,,,,
12346.00,326,1,77183.60,hibernating
12347.00,3,7,4310.00,champions
12348.00,76,4,1797.24,at_Risk
12349.00,19,1,1757.55,promising
12350.00,311,1,334.40,hibernating


In [68]:
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
Recency,4338.00,93.06,100.01,1.00,18.00,51.00,142.75,374.00
Frequency,4338.00,4.27,7.71,1.00,1.00,2.00,5.00,210.00
Monetary,4338.00,2054.27,8989.23,3.75,307.41,674.48,1661.74,280206.02


In [71]:
rfm[["segment", "Recency", "Frequency", "Monetary"]].groupby("segment").agg(["mean", "count"])

Recency       Frequency       Monetary      
                       mean count      mean count     mean count
segment                                                         
about_to_sleep        53.31   352      1.16   352   471.99   352
at_Risk              153.79   593      2.88   593  1084.54   593
cant_loose           132.97    63      8.38    63  2796.16    63
champions              6.36   633     12.42   633  6857.96   633
hibernating          217.61  1071      1.10  1071   488.64  1071
loyal_customers       33.61   819      6.48   819  2864.25   819
need_attention        52.43   187      2.33   187   897.63   187
new_customers          7.43    42      1.00    42   388.21    42
potential_loyalists   17.40   484      2.01   484  1041.22   484
promising             23.51    94      1.00    94   294.01    94

### CHAMPIONS: 
Bu grupta 633 kişi bulunmakta. Ortalama olarak 6 gün önce alışveriş yapılmış. İşlem sıklığı 12. 6857 birim harcama yapılmış. 
Bu grup şirket kazancının çok büyük bir yüzdesini oluşturmaktadır. Amacımız bu grubun memnuniyetini ve mutluluğunu canlı tutmak olmalıdır. Anlamlı günlerde dijital kanallar aracılığıyla bu grubu özel hissettirecek mesajlar atılabilir. Hediyeler gönderilerek özel hissetmeleri sağlanabilir.

### CAN'T LOOSE:

Bu grupta 63kişi bulunmakta. Şirkete 2796.16 gibi yüksek bir kazanç sağlamıştır. Alışveriş sıklığı 8.38 ile yüksek olmasına rağmen ortalama 132 gün önce alışveriş yapılmış. Bu gruptaki kişilerin toplu alışveriş yaptığı varsayılabilir. Bu sınıfa yeni ürün tanıtımları, promosyonlar ve indirimler yapılabilir. Böylelikle etkileşime geçme isteği oluşturulabilir.  

### ABOUT TO SLEEP:

Bu grupta 352 kişi bulunmaktadır. Ortalama 53 gün önce alışveriş yapılmıştır. Bu yüzden kendimizi bu gruba hatırlatmalıyız. Çeşitli kanallar aracılığıyla iletişime geçilip bilgilendirme yapılmalıdır.

Loyal Customers grubundaki müşterileri ilgili birime iletmek için bir excel hazırlayalım.

In [74]:
loyal_df = pd.DataFrame()
loyal_df["loyal_customers"] = rfm[rfm["segment"] == "loyal_customers"].index
loyal_df.head()

,loyal_customers
0,12352.00
1,12359.00
2,12370.00
3,12380.00
4,12388.00


In [76]:
loyal_df.shape

(819, 1)

In [77]:
loyal_df.to_excel("loyal_customers.xlsx")